# Model

intro text: dispatch model

### Dependencies

- `JuMP` for handling optimisation problems
- `Clp` for solving optimisation problems 
- `Plots` for plotting
- `DataFrames` for storing data about components and time series
- `CSV` for reading .csv files

## Basic Structure

table with component and description

might not be necessary

| Component | Description |
| --- | --- |
| bla | bla |

## Mathematical model formulation

The mathematical model formulation is as follows:

$$
\begin{equation}
    \min_{g_{disp,t}; ~cu_{t}; ~d^{stor}_{s, t}; ~l^{stor}_{s, t}} \sum_{disp,t} MC_{disp} g_{disp,t} 
\end{equation}
$$

subject to:

$$
\begin{aligned}
    d_t             & = \sum_{disp} g_{disp, t} & \forall~& t \in T & \text{energy balance} \\
    g_{disp, t}     & \leq G^{max}_{disp, t}    & \forall~& disp \in Disp, t \in T & \text{max generation} \\
    d^{stor}_{s,t}  & \leq G^{max}_{s, t}       & \forall~& s \in S, t \in T & \text{max charge} \\
    l^{stor}_{s, t+1} & = l^{stor}_{s, t} + d^{stor}_{s,t} - \left(\frac{1}{\eta_s}\right) g_{s,t} 
    & \forall~& s \in S, t \in T & \text{storage level}  
\end{aligned}
$$

**Sets**
- Time steps T: $t = {1, ..., 48}$
- Dispatchable generators (including storage) Disp
- NonDisp
- S

**Decision variables:**

- $g_{disp, t} \geq 0$ is the generator dispatch of technology $disp$ in time step $t$
- $cu_{t} \geq 0$ is the curtailment in time step $t$
- $d^{stor}_{s, t} \geq 0$ is the charging of storage $s$ in time step $t$
- $l^{stor}_{s, t} \geq 0$ is the state of charge of storage $s$ in time step $t$


**Parameters:**

- $MC_{disp}$ is the marginal generation cost of technology $disp$
- RES availability
- Installed capacity (generators and storage)
- efficiency


## Julia implementation

For implementing the mathematical model into julia, JuMP is used. It is a domain-specific modeling language for mathematical optimization embedded in Julia.

Before the optimisation problem can be formulated, the necessary input data needs to be loaded.

### Preprocessing

describe data read in


In [ ]:
using JuMP
using Clp
using Plots 
using DataFrames, CSV

In [ ]:
data_path = "data"
time_series = CSV.read(joinpath(data_path, "timedata.csv"),DataFrame)
tech_data = CSV.read(joinpath(data_path, "technologies.csv"),DataFrame)

In [ ]:
T = time_series[:,"hour"]
P = ["coal","gas","pv","wind"]
DISP = ["coal","gas"]
NONDISP = ["pv","wind"]
S = ["PumpedHydro", "Battery"]

DISP = vcat(DISP, S)

tech_data = DataFrames.unstack(tech_data, :technology, :parameter, :value)

In [ ]:
demand = Dict(time_series[:,:hour] .=> time_series[:,:demand])#./1000)

mc = Dict()
g_max = Dict()
eff = Dict()
stor_max = Dict()
for tech in tech_data.technology
    mc[tech] = tech_data[tech_data.technology .== tech, :mc][1]
    g_max[tech] = tech_data[tech_data.technology .== tech, :installed_cap][1]
    eff[tech] = tech_data[tech_data.technology .== tech, :storage_eff][1]
    stor_max[tech] = tech_data[tech_data.technology .== tech, :storage_max][1]
end

pv_installed = g_max["pv"]
wind_installed = g_max["wind"]

avail_pv = Dict(
    ("pv", row["hour"]) => row["pv"] * pv_installed for row in eachrow(time_series)
)
avail_wind = Dict(
    ("wind", row["hour"]) => row["wind"] * wind_installed for row in eachrow(time_series)
)
res_feed_in = merge(avail_pv, avail_wind)

In [ ]:
next_hour(x) = x == T[end] ? T[1] : T[findfirst(isequal(x), T) + 1]

### Modell formulation with `JuMP`

explanin jump part

and highlight solving command

In [ ]:
m = Model(Clp.Optimizer)

In [ ]:
@variables m begin
    G[DISP, T] >= 0
    CU[T] >= 0
    D_stor[S,T] >= 0 # charging/demand from the battery
    L_stor[S,T] >= 0 
end

In [ ]:
@objective(m, Min, sum(mc[disp] * G[disp,t] for disp in DISP, t in T))

In [ ]:
@constraint(m, EnergyBalance[t=T],
    sum(G[disp,t] for disp in DISP)
    + sum(res_feed_in[ndisp,t] for ndisp=NONDISP)
    - sum(D_stor[s,t] for s in S)
    - CU[t]
    == 
    demand[t]
)

@constraint(m, MaxGeneration[disp=DISP, t=T],
    G[disp,t] <= g_max[disp])

@constraint(m, MaxCharge[s=S, t=T],
    D_stor[s,t] <= g_max[s])

@constraint(m, StorageLevel[s=S, t=T],
    L_stor[s, next_hour(t)]
    ==
    L_stor[s, t] 
    + D_stor[s,t] 
    - (1/eff[s]) * G[s,t]
)

In [ ]:
optimize!(m)

### Results and plotting

explain 

In [ ]:
total_cost = objective_value(m)

result_G = value.(G).data

curtailment = value.(CU).data

feedin = [res_feed_in[ndisp,t] for ndisp in NONDISP, t in T]

generation = vcat(feedin, result_G) |> transpose

d = [demand[t] for t in T]

In [ ]:
areaplot(
    generation,
    label=["PV" "Wind" "Coal" "Gas" "PumpedHydro" "Battery"],
    color=[:yellow :lightblue :brown :grey :blue :purple],
    xlabel="Hours",
    ylabel="MW",
    width=0,
    legend=false
)

plot!(d, color=:black, width=3, label = "Demand")
plot!(curtailment .+ d, color=:black, width=2, label="Curtailment", linestyle=:dash)

stor = -value.(D_stor).data |> transpose

areaplot!(
    stor,
    label="",
    color=[:blue :purple],
    width=0
)

hline!([0], color=:black, width=2, label="")

price = dual.(EnergyBalance).data

plot!(
    twinx(),
    price,
    color=:black,
    ylim=(minimum(price)-10,maximum(price)+10),
    width=2,
    leg=false,
    ylabel="Cost per MW",
    linestyle=:dot
)

![Alt text](dispatch_plot.png)